In [1]:
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 53.0 MB/s eta 0:00:00


In [2]:
import torch
import torchaudio
from speechbrain.inference import EncoderDecoderASR
import urllib.request
import os
import requests
import tarfile


INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


In [3]:
# Load pre-trained LLM-based speech recognition model
asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="tmpdir")


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached


hyperparams.yaml:   0%|          | 0.00/4.83k [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch normalizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached


normalizer.ckpt:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch asr.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached


...

INFO:speechbrain.utils.fetching:Fetch lm.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached


lm.ckpt:   0%|          | 0.00/212M [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch tokenizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached


tokenizer.ckpt:   0%|          | 0.00/253k [00:00<?, ?B/s]

INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: normalizer, asr, lm, tokenizer


In [4]:
# Define the URL for LibriSpeech
librispeech_url = "https://www.openslr.org/resources/12/dev-clean.tar.gz"
output_dir = "librispeech_data"
output_file = os.path.join(output_dir, "dev-clean.tar.gz")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Download the dataset
print("Downloading LibriSpeech dataset...")
response = requests.get(librispeech_url, stream=True)

if response.status_code == 200:
    with open(output_file, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Download complete: {output_file}")
else:
    print(f"Failed to download LibriSpeech dataset. HTTP Status: {response.status_code}")

# Extract the tar.gz file
print("Extracting the dataset...")
with tarfile.open(output_file, "r:gz") as tar:
    tar.extractall(path=output_dir)
print(f"Dataset extracted to: {output_dir}")


Download complete: librispeech_data/dev-clean.tar.gz
Extracting the dataset...
Dataset extracted to: librispeech_data


In [5]:
asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="tmpdir")

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch normalizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch asr.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch lm.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch tokenizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: normalizer, asr, lm, tokenizer


In [6]:
# Path to the audio file
audio_file = "librispeech_data/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.flac"

# Load the audio file
waveform, sample_rate = torchaudio.load(audio_file)

# Convert to mono if the waveform has more than one channel
if waveform.shape[0] > 1:  # Check if there are multiple channels
    waveform = torch.mean(waveform, dim=0, keepdim=True)

# Resample the audio if required (ASR model typically expects 16kHz audio)
target_sample_rate = 16000
if sample_rate != target_sample_rate:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
    waveform = resampler(waveform)

# Ensure waveform shape is (time,) by removing extra dimensions
waveform = waveform.squeeze(0)

# Initialize the ASR model
asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="tmpdir")

# Convert waveform to a batch tensor (batch size = 1)
waveform = waveform.unsqueeze(0)  # Add batch dimension (1, time)
wav_lens = torch.tensor([1.0])    # Full length for a single waveform

# Debugging: Print shapes before feeding into the model
print(f"Waveform shape: {waveform.shape}")  # Should be (1, time)
print(f"wav_lens: {wav_lens}")

# Perform speech recognition
transcription = asr_model.transcribe_batch(waveform, wav_lens)

# Print the transcription
print("Transcription:", transcription)


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch normalizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch asr.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch lm.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch tokenizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: normalizer, asr, lm, tokenizer


Waveform shape: torch.Size([1, 93680])
wav_lens: tensor([1.])
Transcription: (['MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL ⁇ '], [[224, 25, 3, 395, 53, 103, 55, 2, 5, 26, 27, 65, 44, 8, 2, 233, 10, 10, 44, 490, 1, 93, 1, 6, 71, 128, 777, 9, 71, 53, 23, 159, 11, 29, 203, 1, 26, 143, 0]])


In [7]:
# Path to the folder containing the audio files
folder_path = "librispeech_data/LibriSpeech/dev-clean/1272/128104"

# Initialize the ASR model
asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="tmpdir")

# Target sample rate for the ASR model
target_sample_rate = 16000

# Iterate through all FLAC files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".flac"):  # Only process FLAC files
        audio_file = os.path.join(folder_path, file_name)
        print(f"Processing file: {audio_file}")

        # Load the audio file
        waveform, sample_rate = torchaudio.load(audio_file)

        # Convert to mono if the waveform has more than one channel
        if waveform.shape[0] > 1:  # Check if there are multiple channels
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        # Resample the audio if required (ASR model typically expects 16kHz audio)
        if sample_rate != target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
            waveform = resampler(waveform)

        # Ensure waveform shape is (time,) by removing extra dimensions
        waveform = waveform.squeeze(0)

        # Convert waveform to a batch tensor (batch size = 1)
        waveform = waveform.unsqueeze(0)  # Add batch dimension (1, time)
        wav_lens = torch.tensor([1.0])    # Full length for a single waveform

        # Perform speech recognition
        transcription = asr_model.transcribe_batch(waveform, wav_lens)

        # Print the transcription
        print(f"Transcription for {file_name}: {transcription}\n")


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch normalizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch asr.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch lm.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch tokenizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: normalizer, asr, lm, tokenizer


Processing file: librispeech_data/LibriSpeech/dev-clean/1272/128104/1272-128104-0002.flac
Transcription for 1272-128104-0002.flac: (['HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND ⁇ '], [[16, 359, 1, 160, 20, 58, 77, 47, 176, 245, 371, 1, 46, 8, 2, 363, 33, 907, 462, 1, 6, 165, 196, 7, 22, 11, 42, 194, 159, 12, 246, 160, 422, 28, 94, 95, 730, 15, 86, 125, 115, 12, 6, 346, 940, 1, 164, 23, 23, 98, 330, 468, 410, 9, 2, 440, 0]])

Processing file: librispeech_data/LibriSpeech/dev-clean/1272/128104/1272-128104-0013.flac
Transcription for 1272-128104-0013.flac: (['MISTER QUILTER HAS MISSED HIS CHANCE FOR HE HAS FAILED EVEN TO MAKE HIMSELF THE TOPPER OF PAINTING ⁇ '], [[224, 25, 3, 395, 53, 103, 193, 247, 4, 29, 789, 34, 16, 193, 827, 4, 242, 9, 300, 54, 130, 2, 9, 134, 25, 8, 127, 40, 7, 12, 0]])

Processing file: librispeech_data/LibriSpeech/dev-clean/1272/128104/1